# Aanvraag / besluit classifier

In [ ]:
# Debug commands to see if Tensorflow GPU is supported
# from keras import backend as K
# print(K.tensorflow_backend._get_available_gpus())

# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

import tensorflow as tf
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
import numpy as np
import time
import math
from PIL import Image
from scipy import misc
import shutil
import keras
import keras.backend as K
import keras_metrics
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.calibration import calibration_curve
import pandas as pd
import os

%matplotlib inline

from keras.preprocessing.image import ImageDataGenerator
 
from examples.aanvraag_besluit.load_data import load_data_aanvraag, preprocess_X
from examples.aanvraag_besluit.transformer import Transformer

from src.stats import list_stats, show_train_curves, show_prediction_list, show_prediction_images
from src.data import split_data
from src.image_display import show_image
from src import models as own_models

# Hot reload packages
%load_ext autoreload
%autoreload 2


In [ ]:
# Set tensorflow GPU memory usage to on the fly rather than preallocate.
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.tensorflow_backend.set_session(tf.Session(config=config))

## Load dataset

In [ ]:
img_dim = (200, 200, 3);
img_dim = (250, 250, 3);
# img_dim = (300, 300, 3);
# img_dim = (400, 400, 3);

[Xtrain_raw, Ytrain_raw, Xvalid_raw, Yvalid_raw] = load_data_aanvraag(
    {
        'images': f'examples/aanvraag_besluit/eerste_dataset/resized/{img_dim[0]}x{img_dim[1]}/',
        'labels': 'examples/aanvraag_besluit/eerste_dataset/labels/'
    },
    {
        'images': f'examples/aanvraag_besluit/tweede_dataset/images/{img_dim[0]}x{img_dim[1]}/',
        'labels': 'examples/aanvraag_besluit/tweede_dataset/labels/'
    },
)

print(f"shape Xtrain[0]: {Xtrain_raw[0].shape}")
print(f"shape Xtrain[1]: {Xtrain_raw[1].shape}")
print(f"shape Ytrain: {Ytrain_raw.shape}")

print(f"shape Xvalid[0]: {Xvalid_raw[0].shape}")
print(f"shape Xvalid[1]: {Xvalid_raw[1].shape}")
print(f"shape Yvalid: {Yvalid_raw.shape}")

In [ ]:
ids = Xvalid_raw[1]['reference'] == 'SU10212124_00001.jpg'
print(Xvalid_raw[1].loc[ids])
Yvalid_raw[ids]

## Preprocess data and labels

In [ ]:
# Preprocess (encode, transform features and labels)
transformer = Transformer()

Xdata_mix = Xtrain_raw[1].append(Xvalid_raw[1])
transformer.fit(Xdata_mix)

Xtrain = preprocess_X(Xtrain_raw[0], Xtrain_raw[1], transformer)
Xvalid = preprocess_X(Xvalid_raw[0], Xvalid_raw[1], transformer)
# print(Xtrain[1][:4])
# print(transformer.decode(Xtrain[1][:4]))
print(Xvalid[1][:4])
print(transformer.decode(Xvalid[1][:4]))

num_features = Xtrain[1].shape[1]
print(Xvalid[1].shape)
assert Xvalid[1].shape[1] == num_features
del Xdata_mix

In [ ]:
# Xtrain[0].nbytes / 1024**2
# Xtrain[0].dtype

In [ ]:
classes = list(set(Ytrain_raw))
print(classes)
num_classes = 2
assert len(classes) == num_classes


print('')
print('--- TRAIN ---')
list_stats(Ytrain_raw)

print('')
print('--- VALID ---')
list_stats(Yvalid_raw)

In [ ]:
# enc = preprocessing.LabelEncoder()  # outputs 1d array, binary classification
# Ytrain = enc.transform(Ytrain_raw)
# Yvalid = enc.transform(Yvalid_raw)

enc = preprocessing.OneHotEncoder()  # outputs 2d array, multi class classification
assert Ytrain_raw.ndim == 1
enc.fit(Ytrain_raw.reshape(-1, 1))

print(Ytrain_raw.shape)
print(Ytrain_raw[:10])

Ytrain = enc.transform(Ytrain_raw.reshape(-1, 1)).toarray()
Yvalid = enc.transform(Yvalid_raw.reshape(-1, 1)).toarray()
print('Ytrain: ', Ytrain.shape)
print('Yvalid: ', Yvalid.shape)
print('Ytrain: ', Ytrain[:10])

# Define model

In [ ]:
# model = own_models.build_multi_feature(num_classes, img_dim, num_features)
model = own_models.create_cnn(img_dim, num_classes)
# model = own_models.create_cnn_deep(img_dim, num_classes)
# model = own_models.create_mlp(num_features, num_classes)

model.summary()

# Train

In [ ]:
batch_size = 20
epochs = 50

run_name = '/cnn_experiment'
LOG_DIR = f'./logs{run_name}'
shutil.rmtree(LOG_DIR, ignore_errors=True)

datagen = ImageDataGenerator(
        zoom_range=0.1,        # randomly zoom into images
        rotation_range=10,      # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,# randomly shift images vertically (fraction of total height)
        shear_range=2.0,  # in degrees
        channel_shift_range=0.1,
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False    # randomly flip images
)

tbCallBack = keras.callbacks.TensorBoard(
    log_dir=LOG_DIR,
    histogram_freq=0,
    write_graph=True,
    write_images=True
)

terminateCB = tf.keras.callbacks.TerminateOnNaN()


def is_binary(model):
    n_classes = model.get_layer('output').output_shape[1]
    return n_classes == 1
    
def compile_model(model):
    assert(K.image_data_format() == 'channels_last')
    
#     if is_binary(model):
#         loss= keras.losses.binary_crossentropy
#     else:
    loss=keras.losses.categorical_crossentropy
    
    model.compile(
        loss=loss,
#         optimizer=keras.optimizers.Adadelta(),
#         optimizer='rmsprop',
#         optimizer='sgd',
#         optimizer=keras.optimizers.SGD(lr=0.01),
#         optimizer=keras.optimizers.Adam(),        
#         optimizer=keras.optimizers.Adam(lr=0.0003),
        optimizer=keras.optimizers.Adam(lr=0.0001),
#         metrics=['accuracy', keras_metrics.recall()]
        metrics=['accuracy', keras_metrics.binary_recall(label=0)]

    )

def train(model, X_train, Y_train, X_test, Y_test, batch_size, epochs):
    compile_model(model)
    
    history = model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(X_test, Y_test),
              callbacks=[tbCallBack, terminateCB]
           )
    return history

def train_gen(model, X_train, Y_train, X_test, Y_test, batch_size, epochs):
    compile_model(model)

#     with tf.Session() as s:
#         s.run(tf.global_variables_initializer())
    history = model.fit_generator(
        datagen.flow(X_train,
                     Y_train,
                     batch_size=batch_size
        ),
        steps_per_epoch=int(np.ceil(X_train.shape[0] / float(batch_size))),
        epochs=epochs,
        validation_data=(X_test, Y_test),
        workers=4,
        callbacks=[tbCallBack, terminateCB]
    )
    return history

t0 = time.time()

# model = own_models.create_cnn_deep_d(img_dim, num_classes)

# Combined data
# history = train(model, Xtrain, Ytrain, Xvalid, Yvalid, batch_size, epochs)

# Meta data
# model = own_models.create_mlp(num_features, num_classes)
# history = train(model, Xtrain[1], Ytrain, Xvalid[1], Yvalid, batch_size, epochs)

# Img data
model = own_models.create_cnn(img_dim, num_classes)
# history = train(model, Xtrain[0], Ytrain, Xvalid[0], Yvalid, batch_size, epochs)
history = train_gen(model, Xtrain[0], Ytrain, Xvalid[0], Yvalid, batch_size, epochs)

show_train_curves(history)

difference = time.time() - t0
print(f'time: {difference} seconds')

In [ ]:
train_score = model.evaluate(Xtrain[0], Ytrain, verbose=1)
print('Train loss:', round(train_score[0], 3))
print(f'Train accuracy: {round(train_score[1] * 100, 2)}%')

valid_score = model.evaluate(Xvalid[0], Yvalid, verbose=1)
print('Test loss:', round(valid_score[0], 3))
valid_acc_str = f'{round(valid_score[1] * 100, 2)}%'
print(f'Test accuracy: {valid_acc_str}')

In [ ]:
print(f"types: {classes}")

print("train predictions, truth")
predictions_train =  model.predict(Xtrain[0], verbose=1)
show_prediction_list(predictions_train, Ytrain)

print("test predictions, truth")
predictions_valid = model.predict(Xvalid[0], verbose=1)
show_prediction_list(predictions_valid, Yvalid)

In [ ]:
idx = 11
id = Xvalid_raw[1]['reference'][idx]
image = Xvalid_raw[0][idx]
print(id)
show_image(image)

In [ ]:
# Y_train_idx = np.argmax(Y_train, axis=1)        
# Y_test_idx = np.argmax(Y_test, axis=1)

# print("train set:")
# show_prediction_images_new(Xtrain, Ytrain_oh, predictions_train, Ytrain_meta, enc, 10)

print("test set:")
show_prediction_images(
    Xvalid_raw[0],
    Yvalid,
    predictions_valid,
    Xvalid_raw[1]['reference'],
    enc,
    300
)

In [ ]:
# def multi_class_to_binary(class_true: np.ndarray, class_pred: np.ndarray):
#     # Converting to probablilty that Y_true == 1
#     assert class_true.shape[1] == 2  # 2 classes
#     assert class_pred.shape[1] == 2  # 2 classes
#     assert class_true.shape[0] == class_pred.shape[0]
    
#     y_true = np.argmax(class_true, axis=1)
    
# #     pred_ids = np.argmax(class_pred, axis=1)
# #     y_prob = class_pred[range(class_pred.shape[0]), pred_ids]
#     y_prob = class_pred[:, 1]
#     assert y_true.shape == y_prob.shape
#     return [y_true, y_prob]
    
# [y_true, y_prob] = multi_class_to_binary(Y_test, predictions_test)


In [ ]:
# print(Yvalid[0:20])
# print(np.round(predictions_valid[0:20], 3))
# predictions_valid.shape
# # print(np.round(y_prob[0:10], 3))

In [ ]:
# # class_pred = predictions_test
# # pred_ids = np.argmax(class_pred, axis=1)
# # y_prob = class_pred[range(class_pred.shape[0]), pred_ids]
# y_prob = predictions_valid

In [ ]:
# # reference https://scikit-learn.org/stable/auto_examples/calibration/plot_calibration_curve.html#sphx-glr-auto-examples-calibration-plot-calibration-curve-py

# n_bins = 5

# print(f'accuracy: {test_acc_str}')
# conf_avg = np.average(predictions_test)
# conf_avg_str = f'{round(conf_avg * 100, 2)}%'
# print(f'confidence avg: {conf_avg_str}')

# def draw_confidence_histogram(y_prob, n_bins):
#     plt.figure()
#     plt.title('Confidence histogram')
#     plt.xlabel("Confidence")
#     plt.ylabel("Sample count")
#     plt.hist(y_prob, bins=n_bins)    
# draw_confidence_histogram(y_prob, n_bins=n_bins)

# def draw_reliability_curve(y_true, y_prob, n_bins):
#     plt.figure()
#     plt.title('Reliability curve')
#     plt.xlabel("Confidence")
#     plt.ylabel("Accuracy")
#     [prob_true_bins, prob_pred_bins] = calibration_curve(y_true, y_prob, n_bins=n_bins)

#     plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
#     plt.plot(prob_pred_bins, prob_true_bins, marker='s')
# draw_reliability_curve(y_true, y_prob, n_bins)

# Thresholding

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, recall_score

conf_threshold = 0.9
labels = ['aanvraag', 'other']

# Stats calculation by hand
# assert len(labels) == 2
# # in binary case confusion matrix is true postive, true negative etc.
# tn, fp, fn, tp = confusion.ravel()
# # Own recall calculation for sanity check
# recall = tp / (tp + fn)
# print(f'recall: {recall}')

# ROC curve
# assert len(labels) == 2  # only works for binary case
# scores = np.where(prob[:,0]>conf_threshold, 0,1)
# print(y_true[:10, :])
# print((predictions_valid[:10, :]).round(2))
# metrics.roc_curve(y_true, scores, pos_label=2)

#
# 
#
def split_bool_arrays(predictions: np.ndarray, threshold, verbose=False):
    assert predictions.shape[1] == 2, 'expecting binary prediction in one hot format'
    
#     y_pred_class = np.argmax(predictions, axis=1)
    y_pred_conf = np.amax(predictions, axis=1)
    
    if verbose:
        print(y_pred_conf.round(2)[:10])
    
    certain = y_pred_conf >= threshold
    uncertain = np.invert(certain)
    return [certain, uncertain]

def split_uncertain(predictions: np.ndarray, threshold, elements, verbose=False):
    """
    Split all elements into certain and uncertain buckets
    
    @return [[elem1_certain, elem1_uncertain], ...]
    """
    for element in elements:
        assert element.shape[0] == predictions.shape[0], 'number of an element not equal to number of predictions'
    
    [certain, uncertain] = split_bool_arrays(predictions, threshold, verbose=verbose)
    
    results = []
    for element in elements:
        certain_bucket = element[certain]
        uncertain_bucket = element[uncertain]
        results.append([certain_bucket, uncertain_bucket])
    return results
    
results = split_uncertain(predictions_valid, conf_threshold, [Xvalid_raw[0], Xvalid_raw[1], Yvalid, predictions_valid])
print('image certain shape: ', results[0][0].shape)
print('image uncertain shape: ', results[0][1].shape)
print('meta certain shape: ', results[1][0].shape)
print('meta uncertain shape: ', results[1][1].shape)

[
    _, # img
    _, # meta
    Yvalid_buckets, # true
    Yvalid_pred_buckets, # prediction
] = results

#


def create_reports(y_true_oh: np.ndarray, y_pred_oh: np.ndarray):
    assert y_true_oh.shape == y_pred_oh.shape
    assert y_true_oh.shape[1] == 2, 'expecting binary one hot inputs'
    y_true = enc.inverse_transform(y_true_oh)
    y_pred = enc.inverse_transform(y_pred_oh)
    
#     confusion = confusion_matrix(y_true, y_pred, labels=labels)
#     plt.imshow(confusion, cmap='binary', interpolation='None')
#     plt.show()
#     print(f'confusion matrix:\n{confusion}')
    y_true_pd = pd.Series(y_true.ravel())
    y_pred_pd = pd.Series(y_pred.ravel())
    crosstab = pd.crosstab(y_true_pd, y_pred_pd, rownames=['True'], colnames=['Predicted'], margins=True)
    
    report = classification_report(y_true, y_pred, target_names=labels)

    return [crosstab, report]


def show_reports(y_true_oh: np.ndarray, y_pred_oh: np.ndarray):
    [crosstab, report] = create_reports(y_true_oh, y_pred_oh)
    print(crosstab)
    
    print()
    print(report)
    

print()
print('--- certain bucket stats ---')
show_reports(Yvalid_buckets[0], Yvalid_pred_buckets[0])

print()
print('--- uncertain bucket stats ---')
show_reports(Yvalid_buckets[1], Yvalid_pred_buckets[1])

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual


class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'


predictions = predictions_valid

def show_results(threshold):
    total = predictions.shape[0]
    [certain, uncertain] = split_bool_arrays(predictions, threshold)
    certain_count = np.sum(certain)
    uncertain_count = np.sum(uncertain)
#     print(certain[:10])
#     print(uncertain[:10])
    
    # Show counts of split
    certain_percentage = certain_count/total*100
    uncertain_percentage = uncertain_count/total*100
    counts_df = pd.DataFrame([
        [certain_count, uncertain_count, total],
        [certain_percentage, uncertain_percentage, 100.0]
    ],
                      columns=['certain', 'uncertain', 'total'],
                      index=['aboslute', 'relative'])
    
    # Show metrics of splits
    [
        Ytrue_buckets,
        Ypred_buckets,
    ] = split_uncertain(predictions, threshold, [Yvalid, predictions_valid])
    
    
    certain_not_empty = Ytrue_buckets[0].shape[0] > 0
    if certain_not_empty:
        y_true = enc.inverse_transform(Ytrue_buckets[0])
        y_pred = enc.inverse_transform(Ypred_buckets[0])
        certain_recall = recall_score(y_true, y_pred, pos_label='aanvraag')    
        print(f'certain examples:\t\t{color.BOLD}{round(certain_percentage, 1)}%{color.END}', end='')
        print(f'\t-> recall: {color.BOLD}{round(certain_recall*100, 2)}%{color.END}')
    else:
        print(f'certain examples:\t\t{color.BOLD}{round(certain_percentage, 1)}%{color.END}')
    print(f'uncertain examples:\t\t{color.BOLD}{round(uncertain_percentage, 1)}%{color.END}')
    
    print()
    print()
    print(counts_df.round(2))
    
    print()
    print()
    print(f'{color.BOLD}## Certain examples{color.END}')
    if certain_not_empty:
        show_reports(Ytrue_buckets[0], Ypred_buckets[0])
    else:
        print('no data')

    print()
    print()
    print(f'{color.BOLD}## Uncertain examples{color.END}')
    if Ytrue_buckets[1].shape[0] == 0:
        print('no data')
    else:
        show_reports(Ytrue_buckets[1], Ypred_buckets[1])

widget = widgets.FloatSlider(
    value=0.9,
    min=0.5,
    max=1.0,
    step=0.005,
    continuous_update=False,
    description='Threshold:',
    readout=True,
    readout_format='.3f',
)

interact(show_results, threshold=widget);

# Output

In [ ]:
def predictions_overview(Y_oh, pred_oh, references, encoder):
    Y_class = encoder.inverse_transform(Y_oh)
    pred_class = encoder.inverse_transform(pred_oh)
    
    data = {'reference': references, 'label': Y_class[:, 0], 'prediction': pred_class[:, 0]}
    df = pd.DataFrame(data)
    return df

DIR = './output'
os.makedirs(DIR, exist_ok=True)

df = predictions_overview(Ytrain, predictions_train, Xtrain_raw[1]['reference'], enc)
df.to_csv(os.path.join(DIR, 'train_predictions.csv'))
print('---TRAIN---')
print(df)

df = predictions_overview(Yvalid, predictions_valid, Xvalid_raw[1]['reference'], enc)
df.to_csv(os.path.join(DIR, 'validation_predictions.csv'))
print('---VALID---')
print(df)

In [ ]:
1.0 + 2.0